In [ ]:
# STEP 1 – Import libraries and load CSV
import numpy as np
import pandas as pd
import nltk

# Load dataset
data = pd.read_csv("imdb.csv",engine='python')



print(data.shape)
print(data.columns)
print(data.head(5))

(50000, 2)
Index(['review', 'sentiment'], dtype='object')
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
x = data["review"]
y = data["sentiment"].str.lower().str.strip().map({'positive': 1, 'negative': 0})

print("Unique labels:", y.unique())
print("Positive reviews:", (y == 1).sum())
print("Negative reviews:", (y == 0).sum())


Unique labels: [1 0]
Positive reviews: 25000
Negative reviews: 25000


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # fixes the LookupError
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Initialize
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define preprocessing function
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)   # keep only letters
    text = text.lower()                      # lowercase
    tokens = word_tokenize(text)             # tokenize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply to all reviews
x_cleaned = x.apply(preprocess_text)

print("Before cleaning:", x.iloc[0])
print("After cleaning:", x_cleaned.iloc[0])


Before cleaning: One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show 

In [ ]:
# STEP 4 – Convert cleaned text to numerical features using Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=5000)
X_bow = vectorizer.fit_transform(x_cleaned).toarray()

print("Shape of Bag of Words:", X_bow.shape)


Shape of Bag of Words: (50000, 5000)


In [ ]:
# STEP 5 – Split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_bow, y, test_size=0.2, random_state=42
)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (40000, 5000)
Testing set size: (10000, 5000)


In [ ]:
# STEP 6 – Train logistic regression model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
# STEP 7 – Evaluate model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8726

Confusion Matrix:
 [[4290  671]
 [ 603 4436]]

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.86      0.87      4961
           1       0.87      0.88      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



In [ ]:
# STEP 8 – Test custom input
sample_review = "This movie was absolutely shit, I hate every moment!"
cleaned = preprocess_text(sample_review)
vectorized = vectorizer.transform([cleaned]).toarray()
prediction = model.predict(vectorized)
print("Predicted Sentiment:", "Positive" if prediction[0] == 1 else "Negative")


Predicted Sentiment: Negative


FileNotFoundError: Cannot find file: bow_nlp.ipynb